[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/FineTuning/blob/main/lessons/module3_advanced/01_lora_qlora.ipynb)

# 🚀 LoRA & QLoRA: Train HUGE Models on Free GPUs!

**Duration:** 1.5 hours  
**Level:** Advanced  
**Prerequisites:** Module 2 complete

---

## Hey! Ready to Train Models with BILLIONS of Parameters? 🤯

Okay, so here's the problem:

**Big models = Better results... but they need MASSIVE GPUs!**

For example:
- GPT-2 (small): 117M parameters → Works on free Colab ✅
- BERT (base): 110M parameters → Works on free Colab ✅
- LLaMA 7B: 7,000M parameters → Needs 28GB GPU 😱
- LLaMA 13B: 13,000M parameters → Needs 52GB GPU 💸💸

**But what if I told you there's a HACK?**

What if you could train a 7B parameter model on a FREE Colab GPU?

**That's LoRA!** (Low-Rank Adaptation)

And **QLoRA** makes it even BETTER (uses even less memory)!

Let me explain this magic trick... 🎩✨

## 🤔 The Problem: Full Fine-Tuning is Expensive

When you normally fine-tune a model, you update **ALL the parameters**.

Imagine a model with 7 billion parameters:
- Each parameter is a number (32-bit float = 4 bytes)
- 7 billion × 4 bytes = 28GB just to LOAD the model
- During training, you need gradients, optimizer states, etc.
- **Total: ~100GB of GPU memory!** 😱

Free Colab GPU? **15GB**. Yeah, not happening.

## 💡 The Solution: Only Train a Tiny Bit!

Here's the genius idea behind LoRA:

**Instead of changing ALL 7 billion parameters, what if we only add and train a few MILLION new parameters?**

Think of it like this:

🏠 **Full Fine-Tuning:** Renovate the entire house  
🛋️ **LoRA:** Just add some new furniture!

You keep the original model **frozen** (don't change it), and add small "adapter" layers that you train.

**Result:**
- Original model: 7B parameters (frozen, no gradients needed)
- LoRA adapters: 4M parameters (trainable)
- Memory needed: **WAY less!**

And the crazy part? **It works just as well as full fine-tuning!** 🤯

## 📚 Quick Theory (I Promise It's Not Boring!)

LoRA works by decomposing weight updates into low-rank matrices.

**In English:**

Instead of updating a huge weight matrix directly:
```
W (original) → W + ΔW (updated)
```

LoRA does:
```
W (frozen) → W + A × B
```

Where:
- `W` = Original weights (frozen, don't train)
- `A` = Small matrix (e.g., 768 × 8)
- `B` = Small matrix (e.g., 8 × 768)
- `A × B` = The update (only 8×768×2 = 12,288 parameters!)

**Example:**
- Original layer: 768 × 768 = 589,824 parameters
- LoRA version: 768 × 8 + 8 × 768 = 12,288 parameters
- **That's 98% fewer parameters!** 🎉

And that "8" is called the **rank** - it's a hyperparameter you can tune!

## 🔥 QLoRA: LoRA on Steroids

QLoRA adds one more trick: **Quantization**

**Normal model:**
- Each number = 32 bits (float32)

**Quantized model:**
- Each number = 4 bits (int4)
- **8x less memory!**

**QLoRA = Quantized LoRA**
- Load model in 4-bit precision
- Add LoRA adapters
- Train only the adapters in full precision

**Result:**
- 7B model in 4-bit: ~3.5GB (instead of 28GB)
- LoRA adapters: ~10MB
- **Total: Fits in free Colab!** 🎊

## 🛠️ Let's Actually Do It!

Time to train a REAL large model on Colab. We'll use a smaller model for this demo (so it runs fast), but the technique works the same for 7B, 13B, or even 70B models!

In [ ]:
# Install the LoRA library (PEFT = Parameter-Efficient Fine-Tuning)
!pip install -q transformers datasets peft accelerate bitsandbytes evaluate

## Step 1: Load Model in 4-bit (QLoRA)

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
import torch
import numpy as np

# Quantization config (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Enable 4-bit loading
    bnb_4bit_quant_type="nf4",             # Use normalized float 4
    bnb_4bit_compute_dtype=torch.float16,   # Compute in fp16
    bnb_4bit_use_double_quant=True,        # Double quantization for extra compression
)

model_name = "bert-base-uncased"  # We'll use BERT for demo (works with any model!)

print("📥 Loading model in 4-bit mode...")
print("   (This is the QLoRA magic!)\n")

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically use GPU if available
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

print("✅ Model loaded!")
print(f"\n💾 Check GPU memory:")
if torch.cuda.is_available():
    print(f"   Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"   Cached: {torch.cuda.memory_reserved()/1024**3:.2f} GB")
else:
    print("   Running on CPU")

## Step 2: Add LoRA Adapters

Now we add the trainable adapter layers!

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=8,                              # Rank (higher = more parameters but better accuracy)
    lora_alpha=16,                    # Scaling factor (usually 2×r)
    target_modules=["query", "value"], # Which layers to add LoRA to
    lora_dropout=0.1,                 # Dropout for regularization
    bias="none",                      # Don't train bias parameters
    task_type=TaskType.SEQ_CLS,      # Sequence classification task
)

print("🔧 Adding LoRA adapters...\n")

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

# See the magic!
model.print_trainable_parameters()

print("\n✨ Look at that! We're only training a TINY fraction of the parameters!")

## 🎓 Understanding the LoRA Config

Let me break down what we just did:

### **r (rank) = 8**
- This is the bottleneck dimension
- Higher = more parameters = better accuracy (but more memory)
- Typical values: 4, 8, 16, 32
- Start with 8, increase if needed

### **lora_alpha = 16**
- Scaling factor for the LoRA updates
- Rule of thumb: 2 × r
- Affects how much the adapters influence the output

### **target_modules = ["query", "value"]**
- Which attention layers to add LoRA to
- Common choices:
  - `["query", "value"]` - Minimal (fastest)
  - `["query", "key", "value"]` - Balanced
  - `["query", "key", "value", "output"]` - Maximum (slowest)

### **lora_dropout = 0.1**
- Regularization to prevent overfitting
- 0.05 to 0.1 is typical

## Step 3: Prepare Data (Same as Before!)

In [ ]:
from datasets import load_dataset
import evaluate

# Load dataset
print("📥 Loading dataset...")
dataset = load_dataset('imdb')

# Use subset for quick demo
small_train = dataset['train'].shuffle(seed=42).select(range(1000))
small_test = dataset['test'].shuffle(seed=42).select(range(200))

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

train_dataset = small_train.map(tokenize_function, batched=True)
test_dataset = small_test.map(tokenize_function, batched=True)

# Metrics
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print("✅ Data ready!")

## Step 4: Train! (It's FAST!)

In [ ]:
# Training arguments (optimized for LoRA)
training_args = TrainingArguments(
    output_dir='./lora_results',
    
    # LoRA can use HIGHER learning rates!
    learning_rate=1e-4,  # 10x higher than normal fine-tuning!
    
    # Batch sizes (can be larger with LoRA)
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    
    # Epochs
    num_train_epochs=3,
    
    # Evaluation
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    
    # Logging
    logging_steps=50,
    
    # Save space
    save_total_limit=2,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

print("🚀 Starting LoRA training...\n")
print("Notice how FAST this is compared to full fine-tuning!\n")

# Train!
result = trainer.train()

print("\n✅ Training complete!")
print(f"\n⏱️ Time: {result.metrics['train_runtime']:.1f} seconds")
print(f"🎯 Final loss: {result.training_loss:.4f}")

## Step 5: Evaluate & Compare

In [ ]:
# Evaluate
eval_results = trainer.evaluate()

print("📊 EVALUATION RESULTS\n" + "="*50)
print(f"\n🎯 Accuracy: {eval_results['eval_accuracy']:.3f}")
print(f"📉 Loss: {eval_results['eval_loss']:.3f}")

print("\n💡 Fun Fact:")
print("   This accuracy is usually JUST AS GOOD as full fine-tuning!")
print("   But we trained 100x fewer parameters! 🤯")

## 💾 Saving Your LoRA Model

Here's the BEST part: LoRA adapters are TINY!

In [ ]:
import os

# Save LoRA adapters
output_dir = "./my_lora_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to {output_dir}\n")

# Check file sizes
print("📦 File sizes:")
for file in os.listdir(output_dir):
    filepath = os.path.join(output_dir, file)
    if os.path.isfile(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"   {file}: {size_mb:.2f} MB")

print("\n🤯 The adapter_model.bin is TINY!")
print("   Full model would be 400+ MB")
print("   LoRA adapters are just a few MB!")
print("\n   You can share this on GitHub, email it, whatever!")

## 🔄 Loading Your LoRA Model Later

In [ ]:
from peft import PeftModel

# To load later:
print("📥 How to load your LoRA model:\n")
print("```python")
print("# 1. Load base model")
print("base_model = AutoModelForSequenceClassification.from_pretrained(")
print("    'bert-base-uncased',")
print("    quantization_config=bnb_config")
print(")")
print("")
print("# 2. Load LoRA adapters")
print("model = PeftModel.from_pretrained(base_model, './my_lora_model')")
print("")
print("# 3. Use it!")
print("predictions = model(inputs)")
print("```")

print("\n✨ That's it! Super simple!")

## 🎯 LoRA vs Full Fine-Tuning Comparison

Let me show you the differences:

| Aspect | Full Fine-Tuning | LoRA | QLoRA |
|--------|------------------|------|-------|
| **Trainable Params** | 100% (110M) | ~1% (1M) | ~1% (1M) |
| **GPU Memory** | ~40GB | ~15GB | ~8GB |
| **Training Speed** | Baseline | 1.5x faster | 1.2x faster |
| **Saved Model Size** | 440MB | 4MB | 4MB |
| **Accuracy** | 100% | 98-100% | 97-100% |
| **Learning Rate** | 3e-5 | 1e-4 (higher!) | 1e-4 |
| **Best For** | Small models | Large models | HUGE models |

**TL;DR:**
- LoRA/QLoRA give you 97-100% of the accuracy
- With 1% of the trainable parameters
- Using way less memory
- Training faster
- Producing tiny model files

**It's basically magic.** ✨

## 🔧 Hyperparameter Tuning for LoRA

Here's what to adjust:

### **Rank (r)**
```python
r=4   # Minimal - fastest, smallest, might lose some accuracy
r=8   # Sweet spot - balanced
r=16  # More capacity - slower, bigger, sometimes better
r=32  # Overkill for most tasks
```

**Rule:** Start with 8. Increase if accuracy is too low.

### **Alpha (lora_alpha)**
```python
lora_alpha = 2 * r  # Common rule of thumb
```

### **Target Modules**
```python
# Minimal (fastest)
target_modules = ["q_proj", "v_proj"]

# Balanced
target_modules = ["q_proj", "k_proj", "v_proj"]

# Maximum (most trainable params)
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
```

**Note:** Module names change per model architecture. Check model config!

### **Learning Rate**
```python
# LoRA can handle HIGHER learning rates!
learning_rate = 1e-4  # vs 3e-5 for full fine-tuning
learning_rate = 2e-4  # Sometimes even this works!
```

## 🌟 Real-World LoRA Applications

People are using LoRA to train:

### **1. LLaMA/Mistral Models (7B-70B parameters)**
```python
# This would need 280GB without LoRA!
# With QLoRA? Fits in 24GB!
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b",
    quantization_config=bnb_config,
    device_map="auto"
)
```

### **2. Multiple LoRA Adapters for Different Tasks**
```python
# Load base model once
base_model = load_base_model()

# Switch between tasks by swapping adapters!
sentiment_model = PeftModel.from_pretrained(base_model, './sentiment_lora')
summary_model = PeftModel.from_pretrained(base_model, './summary_lora')
qa_model = PeftModel.from_pretrained(base_model, './qa_lora')
```

### **3. Personalized AI Assistants**
- Train one LoRA adapter per user
- Each adapter is tiny (4-10MB)
- Share the base model, swap adapters per user
- Suddenly you can serve THOUSANDS of personalized models!

## 🎓 Key Takeaways

1. **LoRA = Low-Rank Adaptation**
   - Only train a tiny fraction of parameters
   - Get 98%+ of full fine-tuning performance
   - Use way less memory

2. **QLoRA = Quantized LoRA**
   - Load base model in 4-bit
   - Use even LESS memory
   - Train huge models on free GPUs!

3. **When to Use LoRA:**
   - Training models with >1B parameters
   - Limited GPU memory
   - Need to save/share small model files
   - Want faster training
   - Multiple tasks with same base model

4. **When to Use Full Fine-Tuning:**
   - Small models (<500M parameters)
   - You have lots of GPU memory
   - Need absolute maximum accuracy
   - Simple single-task use case

5. **LoRA Hyperparameters:**
   - `r=8`: Good default rank
   - `lora_alpha=16`: Usually 2×r
   - `target_modules`: Start with ["q", "v"]
   - `learning_rate=1e-4`: Higher than normal!

## 🎉 You're Now a LoRA Expert!

You just learned the technique that's revolutionizing AI fine-tuning!

**This is how people are fine-tuning:**
- LLaMA 2 (7B-70B)
- Mistral (7B)
- Falcon (40B)
- All the latest open-source LLMs!

**On consumer GPUs!** Even free ones!

This is a game-changer. You can now compete with big tech companies using just a free Colab notebook.

Pretty awesome, right? 🚀

---

**Next up:** Multi-Task Learning - Train one model for multiple jobs! 🎨

## 📚 Resources & Further Reading

**Papers:**
- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

**Libraries:**
- [PEFT (Parameter-Efficient Fine-Tuning)](https://github.com/huggingface/peft)
- [bitsandbytes (Quantization)](https://github.com/TimDettmers/bitsandbytes)

**Community:**
- [HuggingFace Forums - PEFT](https://discuss.huggingface.co/c/peft/)
- [r/LocalLLaMA](https://reddit.com/r/LocalLLaMA) - Lots of LoRA discussions!

Now go train some HUGE models! 💪